In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("tor.csv")

In [4]:
df.head()

,TransactionId,UserId,VisitYear,VisitMonth,VisitMode,AttractionId,Rating,ContinentId,RegionId,CountryId,CityId,Continent,Region,Country,CityName,Attraction,AttractionAddress,AttractionTypeId,AttractionType,VisitModeName
0,8,7567,2022,10,4,640,5,2,8,48,464,America,Northern America,Canada,Ontario,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Friends
1,69,9804,2022,7,2,640,4,4,15,109,535,Australia & Oceania,Australia,Australia,Sydney,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
2,232,4371,2020,3,2,640,5,4,15,109,1434,Australia & Oceania,Australia,Australia,Melbourne,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
3,300,11099,2020,2,2,640,5,5,21,163,780,Europe,Western Europe,United Kingdom,Brighton,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Couples
4,376,5021,2020,1,4,640,4,4,15,109,1434,Australia & Oceania,Australia,Australia,Melbourne,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,Friends


In [5]:
df.count()

TransactionId        18884
UserId               18884
VisitYear            18884
VisitMonth           18884
VisitMode            18884
AttractionId         18884
Rating               18884
ContinentId          18884
RegionId             18884
CountryId            18884
CityId               18884
Continent            18884
Region               18884
Country              18884
CityName             18884
Attraction           18884
AttractionAddress    18884
AttractionTypeId     18884
AttractionType       18884
VisitModeName        18884
dtype: int64

In [6]:
df.isnull().sum()

TransactionId        0
UserId               0
VisitYear            0
VisitMonth           0
VisitMode            0
AttractionId         0
Rating               0
ContinentId          0
RegionId             0
CountryId            0
CityId               0
Continent            0
Region               0
Country              0
CityName             0
Attraction           0
AttractionAddress    0
AttractionTypeId     0
AttractionType       0
VisitModeName        0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [9]:
#IMPORTING LIBRARIES FOR SPLITING AND EVALUATING THE MODEL
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [9]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [11]:
selected_features = [      
    "VisitModeName",    
    "AttractionId",     
    "Attraction",      
    "AttractionType",  
    "CountryId",        
    "RegionId"          
]

In [12]:
categorical_features = ["VisitModeName", "AttractionType"]

df_selected = df[selected_features + ["Rating"]].copy()

ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_features = ohe.fit_transform(df_selected[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(categorical_features))

bool_cols = df_selected.select_dtypes(include=["bool"]).columns
df_selected[bool_cols] = df_selected[bool_cols].astype(int)

target_enc = TargetEncoder()
df_selected["Attraction"] = target_enc.fit_transform(df_selected["Attraction"], df_selected["Rating"])

df_selected = df_selected.drop(columns=categorical_features)
df_selected = pd.concat([df_selected, encoded_df], axis=1)

In [13]:
df_selected.head()

,AttractionId,Attraction,CountryId,RegionId,Rating,VisitModeName_Business,VisitModeName_Couples,VisitModeName_Family,VisitModeName_Friends,VisitModeName_Solo,...,AttractionType_National Parks,AttractionType_Nature & Wildlife Areas,AttractionType_Neighborhoods,AttractionType_Points of Interest & Landmarks,AttractionType_Religious Sites,AttractionType_Spas,AttractionType_Speciality Museums,AttractionType_Volcanos,AttractionType_Water Parks,AttractionType_Waterfalls
0,640,4.314734,48,8,5,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,640,4.314734,109,15,4,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,640,4.314734,109,15,5,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,640,4.314734,163,21,5,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,640,4.314734,109,15,4,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
x = df_selected.drop(columns=["Rating"])
y = df_selected["Rating"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
rf_model = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_estimators=50, random_state=42)

In [17]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [18]:
import numpy as np

In [20]:
rf_pred = rf_model.predict(X_test)
rf_pred = np.round(rf_pred).astype(int)
rf_pred = np.clip(rf_pred, 1, 5) 

rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

In [21]:
print(rf_mae)

0.7352396081546201


In [22]:
print(rf_rmse)

0.997348881867871


In [23]:
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [24]:
model_pred = model.predict(X_test)
model_pred = np.round(model_pred).astype(int)
model_pred = np.clip(model_pred, 1, 5) 

model_mae = mean_absolute_error(y_test, model_pred)
model_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))

In [25]:
print(model_mae)
print(model_rmse)

0.7415938575589092
0.997348881867871


In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}

xgb = XGBRegressor(random_state=42)
random_search = RandomizedSearchCV(xgb, param_distributions=param_grid, n_iter=20, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
random_search.fit(X_train, y_train)

best_xgb = random_search.best_estimator_

In [28]:
y_pred = best_xgb.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [29]:
print(mae,rmse)

0.7632330060005188 0.9674521643372939


In [30]:
import joblib

In [31]:
joblib.dump(best_xgb, r"D:\Tourism\Rating model.pkl")

['D:\\Tourism\\Rating model.pkl']

In [32]:
joblib.dump(ohe, r"D:\Tourism\ohe.pkl")

['D:\\Tourism\\ohe.pkl']

In [33]:
joblib.dump(target_enc, r"D:\Tourism\target_enc.pkl")

['D:\\Tourism\\target_enc.pkl']